<a href="https://colab.research.google.com/github/tsenga2/keio-quant-macro/blob/spring23/obtain_bcycle_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import datetime

start_date = datetime.datetime(1976, 1, 1)
end_date = datetime.datetime(2022, 12, 31)

series_ids = {
    'Y': 'GDPCA',
    'C': 'PCECCA',
    'I': 'GPDICA',
    'N': 'HOANBS'
}

data = {}
for variable, series_id in series_ids.items():
    data[variable] = pdr.get_data_fred(series_id, start=start_date, end=end_date)

df = pd.concat(data, axis=1)

print("Original DataFrame:")
print(df.head())

# Convert index to datetime if it's not already
df.index = pd.to_datetime(df.index)

# Function to annualize the data
def annualize_data(df):
    # For Y, C, and I, take the first quarter value of each year
    annual_yCi = df[['Y', 'C', 'I']].resample('Y').first()

    # For N, calculate the annual average
    annual_N = df['N'].resample('Y').mean()

    # Combine the annualized data
    annual_df = pd.concat([annual_yCi, annual_N], axis=1)

    return annual_df

# Create the annualized DataFrame
annual_df = annualize_data(df)

# Calculate logarithms of the series
log_df = np.log(annual_df)
print(log_df.tail)

# Apply HP filter to the log series with lamb=100
filtered_data = {}
for variable in log_df.columns:
    cycle, trend = hpfilter(log_df[variable], lamb=100)
    filtered_data[variable] = cycle

filtered_df = pd.concat(filtered_data, axis=1)

# Rename columns in filtered_df
filtered_df.columns = ['Y', 'C', 'I', 'N']

# Calculate business cycle statistics
std_dev = filtered_df.std() * 100
relative_std_dev = std_dev / std_dev['Y']
correlations = filtered_df.corr()['Y']

# Create a DataFrame with the calculated statistics
statistics_data = {
    'σ(x) (%)': std_dev,
    'σ(x)/σ(Y)': relative_std_dev,
    'Corr(x, Y)': correlations
}

statistics_df = pd.DataFrame(statistics_data)

# Format the DataFrame
formatted_df = statistics_df.applymap(lambda x: '{:.3f}'.format(x))

# Add a header row
header = pd.DataFrame({'U.S. Data, 1976 to 2022': ['σ(x) (%)', 'σ(x)/σ(Y)', 'Corr(x, Y)']}).T
header.columns = formatted_df.columns

# Concatenate the header and formatted DataFrame
result = pd.concat([header, formatted_df])

# Print the resulting table
print("\nBusiness Cycle Statistics:")
print(result)

Original DataFrame:
                     Y           C          I         N
                 GDPCA      PCECCA     GPDICA    HOANBS
DATE                                                   
1976-01-01 6387.437000 4050.586000 758.582000 59.191000
1976-04-01         NaN         NaN        NaN 59.224000
1976-07-01         NaN         NaN        NaN 59.461000
1976-10-01         NaN         NaN        NaN 59.780000
1977-01-01 6682.804000 4221.781000 866.804000 60.412000
<bound method NDFrame.tail of             (Y, GDPCA)  (C, PCECCA)  (I, GPDICA)   HOANBS
DATE                                                     
1976-12-31    8.762088     8.306617     6.631451 4.084530
1977-12-31    8.807293     8.348012     6.764813 4.123171
1978-12-31    8.861167     8.390846     6.874272 4.174010
1979-12-31    8.892336     8.414338     6.908721 4.208948
1980-12-31    8.889765     8.411211     6.802673 4.200988
1981-12-31    8.914826     8.424982     6.887032 4.209082
1982-12-31    8.896631     8.439514   

<ipython-input-42-e2fbeeac53fb>:74: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  formatted_df = statistics_df.applymap(lambda x: '{:.3f}'.format(x))
